<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#SqlAlchemy-and-ORMs" data-toc-modified-id="SqlAlchemy-and-ORMs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><code>SqlAlchemy</code> and ORMs</a></span></li><li><span><a href="#Creating-an-engine-and-basic-queries" data-toc-modified-id="Creating-an-engine-and-basic-queries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating an engine and basic queries</a></span></li><li><span><a href="#Direct-pandas-interface" data-toc-modified-id="Direct-pandas-interface-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Direct <code>pandas</code> interface</a></span></li></ul></div>

# Introduction

Often analysts will wish to bring the results of `SQL` queries over into a scripting data analysis language like `Python` or `R`. Both of these languages (and indeed all modern programming languages) offer packages, modules or libraries that let you connect to `SQL` databases.

That being said, however, it is wise to devolve as much of the data analysis effort as possible to the database you are working with. Modern `SQL` engines are very efficient, and can often handle moderately 'big data' (i.e. data that doesn't fit in your computer memory) much more effectively than can either `Python` or `R`. In fact, a common 'trick' for large scale data analysis in `Python` is to devolve analysis to a local `sqlite` database (have a read of **[this walkthrough](https://www.pythonforengineers.com/open-a-5-gb-file-in-python/)**)

# `SqlAlchemy` and ORMs

If your work involves the full range of `CRUD` (create, read, update and delete) operations on a database, it will make sense for you to use a full **object relational mapper (ORM)** library, for example `SqlAlchemy`. An ORM lets you create `Python` classes that can be mapped directly over to tables in `SQL`. Each **instance** of a `Python` class then maps over to a **row** in a `SQL` table. The nice thing about `SqlAlchemy` and ORMs in general is that you have to learn just one workflow: that of the ORM itself. The ORM then handles the details of the connection to particular databases, slight differences in `SQL` syntax and functionality etc. 

For lighter **read-only** work (this is the majority of data analysis work) you may prefer to use a package dedicated to a particular database. For `PostgreSQL` a common choice in `Python` is `psycopg2`.

We'll use `SqlAlchemy` below to see the workflow, but you will have to install `psycopg2` too, as `SqlAlchemy` will use that package to create a connection. At your command line run

```
pip install psycopg2-binary
```

This installs the precompiled **binary executable**, just to get us started. Note that for serious work, however, you should compile `psycopg2` for your system as it will provide better performance: see the **[installation instructions](https://www.psycopg.org/docs/install.html)**.

# Creating an engine and basic queries

Firstly, we need to create an `engine` in `SqlAlchemy`. This is an object we use to run queries. We need to pass in a **connection string** to do this. Connection strings have the general form

**DB_TYPE**+**DB_CONNECTOR**://**USERNAME**:**PASSWORD**@**HOST**:**PORT**/**DB_NAME**

Here are some options for the **DB_TYPE** and **DB_CONNECTOR** parameters, depending on which database you are working with. Note that in most cases you will also have to install the relevant connector package before use with `SqlAlchemy` (we installed `psycopg2` above).

| Database | DB_TYPE | DB_CONNECTORS |
|--|--|--|
| MySQL | `mysql` | `pymysql`, `mysqldb` |
| PostgreSQL | `postgresql` | `psycopg2`, `pg8000` |
| Oracle | `oracle` | `cx_oracle` |
| Microsoft SQL | `mssql` | `pymssql`, `pyodbc` |
| SQLite | `sqlite` | None required (built-in) |  

Parameters **USERNAME**, **PASSWORD**, **HOST**, **PORT** and **DB_NAME** should be familiar from connecting with `DBeaver` earlier in the course.

Let's create an engine connecting to the `omni_pool` database. We will set argument `echo=True` so you can see what `SqlAlchemy` is doing in the background as we work with it. For real use, you will probably not wish to set this argument. We also set `pool_size=1` to limit the number of connections to the database in the *connection pool* to one.

In [1]:
from sqlalchemy import create_engine, text
engine = create_engine(
    "postgresql+psycopg2://"\
    "omni_user:niytna5mcwsz124i@"\
    "db-postgresql-lon1-20135-do-user-7581050-0.b.db.ondigitalocean.com:25061/omni_pool",
    echo=True,
    pool_size=1
)

So far, so good. Let's now use our engine to run a query. First we need to acquire a connection from the engine. Typically in `Python` we do this within a `with` block: this will make sure that, whatever happens, the connection to the database is returned to the pool at the end of execution of the block.

In [2]:
with engine.connect() as connection:
    results = connection.execute(text("SELECT * FROM teams"))

2021-05-27 18:40:55,501 INFO sqlalchemy.engine.Engine select version()
2021-05-27 18:40:55,502 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-27 18:40:55,551 INFO sqlalchemy.engine.Engine select current_schema()
2021-05-27 18:40:55,552 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-27 18:40:55,599 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-05-27 18:40:55,600 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-27 18:40:55,656 INFO sqlalchemy.engine.Engine SELECT * FROM teams
2021-05-27 18:40:55,657 INFO sqlalchemy.engine.Engine [generated in 0.00120s] {}


Note that we see the background operations, having set `echo=True` above. What `type()` is the `results` object returned

In [3]:
type(results)

sqlalchemy.engine.cursor.LegacyCursorResult

Depending on your version of `SqlAlchemy`, this will either be a type called a `ResultProxy` or a `LegacyCursorResult` One of the nice things about working with an ORM is that the objects it provides have a host of methods and properties, for example:

In [4]:
results.rowcount

10

We see there are 10 rows in this set of results. How do we actually get the data? Let's try just calling the object itself

In [5]:
results

Hmm, it just prints the object. To see the results, we have to tell `SqlAlchemy` to actually go and fetch them from the database. We have a range of `.fetch___()` methods available. Here we'll use `.fetchall()` to get **all** results

In [6]:
rows = results.fetchall()
rows

[(1, 'Audit Team 1', '30'),
 (2, 'Audit Team 2', '30'),
 (3, 'Risk Team 1', '50'),
 (4, 'Risk Team 2', '50'),
 (5, 'Audit Escalate', '60'),
 (6, 'Risk Escalate', '100'),
 (7, 'Data Team 1', '40'),
 (8, 'Data Team 2', '40'),
 (9, 'Data Escalate', '80'),
 (10, 'Corporate', '100')]

The objects in the `list` look like `tuple`s. Let's pull out the first of them

In [7]:
first_row = rows[0]
first_row

(1, 'Audit Team 1', '30')

This looks like a `tuple`, but it isn't really: in fact, it's another dedicated `SqlAlchemy` type

In [8]:
type(first_row)

sqlalchemy.engine.row.LegacyRow

In particular, we can perform operations on this type of object that look more suited for a `dictionary`

In [9]:
first_row['name']

'Audit Team 1'

In [10]:
first_row.keys()

RMKeyView(['id', 'name', 'charge_cost'])

Let's create a list of `dictionaries` from these `RowProxy` objects. We can do this straightforwardly using a `list comprehension`

In [11]:
rows = [dict(row) for row in rows]
rows

[{'id': 1, 'name': 'Audit Team 1', 'charge_cost': '30'},
 {'id': 2, 'name': 'Audit Team 2', 'charge_cost': '30'},
 {'id': 3, 'name': 'Risk Team 1', 'charge_cost': '50'},
 {'id': 4, 'name': 'Risk Team 2', 'charge_cost': '50'},
 {'id': 5, 'name': 'Audit Escalate', 'charge_cost': '60'},
 {'id': 6, 'name': 'Risk Escalate', 'charge_cost': '100'},
 {'id': 7, 'name': 'Data Team 1', 'charge_cost': '40'},
 {'id': 8, 'name': 'Data Team 2', 'charge_cost': '40'},
 {'id': 9, 'name': 'Data Escalate', 'charge_cost': '80'},
 {'id': 10, 'name': 'Corporate', 'charge_cost': '100'}]

We can then create a `DataFrame` from this `list` of `dictionaries`. One of the nice things about `pandas` is that it supports many ways in which to construct `DataFrame`s!

In [12]:
import pandas as pd
results = pd.DataFrame(rows)
results.columns = first_row.keys()
results

,id,name,charge_cost
0,1,Audit Team 1,30
1,2,Audit Team 2,30
2,3,Risk Team 1,50
3,4,Risk Team 2,50
4,5,Audit Escalate,60
5,6,Risk Escalate,100
6,7,Data Team 1,40
7,8,Data Team 2,40
8,9,Data Escalate,80
9,10,Corporate,100


# Direct `pandas` interface

That was pretty straightforward, but `pandas` also provides the `.read_sql()` function to create a `DataFrame` directly from a query. We need to pass in two arguments: the `SQL` query to run, and the connection to run it with.

In [13]:
sql_query = """
    SELECT 
      department,
      ROUND(AVG(salary), 2) AS avg_salary,
      COUNT(id) AS num_employees
    FROM employees 
    GROUP BY department"""

with engine.connect() as connection:
    results = pd.read_sql(sql = sql_query, con = connection)
results

2021-05-27 18:40:56,185 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-05-27 18:40:56,186 INFO sqlalchemy.engine.Engine [generated in 0.00101s] {'name': '\n    SELECT \n      department,\n      ROUND(AVG(salary), 2) AS avg_salary,\n      COUNT(id) AS num_employees\n    FROM employees \n    GROUP BY department'}
2021-05-27 18:40:56,244 INFO sqlalchemy.engine.Engine 
    SELECT 
      department,
      ROUND(AVG(salary), 2) AS avg_salary,
      COUNT(id) AS num_employees
    FROM employees 
    GROUP BY department
2021-05-27 18:40:56,246 INFO sqlalchemy.engine.Engine [raw sql] {}


,department,avg_salary,num_employees
0,Marketing,61132.48,84
1,Training,59852.68,81
2,Research and Development,58450.00,94
3,Business Development,61175.28,77
4,Sales,62600.53,80
5,Product Management,55723.27,79
6,Support,59573.90,81
7,Legal,56503.95,102
8,Accounting,60557.16,72
9,Human Resources,60200.30,90


Again, however, it makes sense to try to devolve as much of your analysis as possible to the database. In particular, if you are performing **split-apply-combine** operations on big data sets, try to perform these as far as possible in `SQL`, so that you are bringing only **summarised/reduced** data back into `pandas`. 

Often, however, you may be limited to **read-only** access to a database, in which case you may have little choice but to perform certain operations in `pandas`, or alternatively, for large datasets, in a local `sqlite` database as mentioned above.

<hr style="border:8px solid black"> </hr>

***

**<u>Task - 5 mins</u>**

* Fetch all the rows from the `teams` table in the `omni_pool` database and save them to a new `DataFrame` called `teams`
* [**Harder**] Get a `DataFrame` called `team_avg_salaries` showing team name (`team_name`) with the average salary (`avg_salary`) for each team. This will require joining and aggregation: perform both in the database.

**Solution**

In [14]:
with engine.connect() as connection:
    teams = pd.read_sql(sql='SELECT * FROM teams', con=connection)
teams

2021-05-27 18:40:56,312 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-05-27 18:40:56,313 INFO sqlalchemy.engine.Engine [cached since 0.1278s ago] {'name': 'SELECT * FROM teams'}
2021-05-27 18:40:56,374 INFO sqlalchemy.engine.Engine SELECT * FROM teams
2021-05-27 18:40:56,376 INFO sqlalchemy.engine.Engine [raw sql] {}


,id,name,charge_cost
0,1,Audit Team 1,30
1,2,Audit Team 2,30
2,3,Risk Team 1,50
3,4,Risk Team 2,50
4,5,Audit Escalate,60
5,6,Risk Escalate,100
6,7,Data Team 1,40
7,8,Data Team 2,40
8,9,Data Escalate,80
9,10,Corporate,100


In [15]:
with engine.connect() as connection:
    team_avg_salaries = pd.read_sql(sql='''
            SELECT 
                t.name AS team_name,
                ROUND(AVG(e.salary), 2) AS avg_salary
            FROM employees AS e INNER JOIN teams AS t
            ON e.team_id = t.id
            GROUP BY t.name
        ''',
    con=connection
    )

team_avg_salaries

2021-05-27 18:40:56,438 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-05-27 18:40:56,439 INFO sqlalchemy.engine.Engine [cached since 0.2542s ago] {'name': '\n            SELECT \n                t.name AS team_name,\n                ROUND(AVG(e.salary), 2) AS avg_salary\n            FROM employees AS e INNER JOIN teams AS t\n            ON e.team_id = t.id\n            GROUP BY t.name\n        '}
2021-05-27 18:40:56,502 INFO sqlalchemy.engine.Engine 
            SELECT 
                t.name AS team_name,
                ROUND(AVG(e.salary), 2) AS avg_salary
            FROM employees AS e INNER JOIN teams AS t
            ON e.team_id = t.id
            GROUP BY t.name
        
2021-05-27 18:40:56,503 INFO sqlalchemy.engine.Engine [raw sql] {}


,team_name,avg_salary
0,Audit Team 2,63909.17
1,Risk Team 1,63054.65
2,Data Team 1,62983.22
3,Data Team 2,59545.49
4,Corporate,60988.89
5,Audit Escalate,59210.88
6,Risk Team 2,56816.13
7,Data Escalate,59944.80
8,Audit Team 1,57203.59
9,Risk Escalate,56514.34


***

<hr style="border:8px solid black"> </hr>